In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install fastai==0.7.0
!pip install torchtext==0.2.3
import os

os.makedirs("data\store",exist_ok=True)
os.makedirs(".kaggle",exist_ok=True)
!pip install kaggle
#!mkdir .kaggle

import json 
token = {"username":"hduongck","key":"983e2ab1fbb29cf2734bcbf8811d42fb"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
 
!chmod 600 /content/.kaggle/kaggle.json

!kaggle config set -n path -v{/content/data}

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle competitions download -c demand-forecasting-kernels-only -p /content/data/store

In [0]:
!pip install parfit

  Stored in directory: /root/.cache/pip/wheels/d9/8a/c9/ed86e85ad0cb41ce2fdcb6bc215902dc475bd3fb1fef7c742d
Successfully built parfit


In [0]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from IPython.display import display

In [0]:
PATH ='data/store/'
!ls {PATH}

sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [0]:
!unzip /content/data/store/\*.zip -d data/store

Archive:  /content/data/store/test.csv.zip
  inflating: data/store/test.csv     

Archive:  /content/data/store/sample_submission.csv.zip
  inflating: data/store/sample_submission.csv  

Archive:  /content/data/store/train.csv.zip
  inflating: data/store/train.csv    

3 archives were successfully processed.


# Data interpretation


In [0]:
table_names = ['train','test']

In [0]:
tables = [pd.read_csv(f'{PATH}{fname}.csv',low_memory=True) for fname in table_names]

In [0]:
for t in tables : display(t.head())

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [0]:
train,test = tables

In [0]:
train.date = train.date.astype('datetime64[ns]')

In [0]:
train.dtypes

date     datetime64[ns]
store             int64
item              int64
sales             int64
dtype: object

# Data preprocessing

In [0]:
val_idx = np.flatnonzero((train.date<=datetime.datetime(2017,12,31))&(train.date>=datetime.datetime(2017,10,1)))

In [0]:
trn_idx = np.flatnonzero((train.date<datetime.datetime(2017,10,1)))

In [0]:
trn_idx[:20]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [0]:
add_datepart(train,'date')

In [0]:
add_datepart(test,'date')

In [0]:
train.shape

(913000, 16)

In [0]:
train_split = train.iloc[trn_idx,:].copy()

In [0]:
valid = train.iloc[val_idx,:].copy()

In [0]:
len(valid)

46000

In [0]:
X_train,y_train,nas=proc_df(train_split,'sales')

In [0]:
X_train.shape

(867000, 15)

In [0]:
y_train = np.log1p(np.clip(y_train,0,None))

In [0]:
X_valid,y_valid,nas = proc_df(valid,'sales',nas)

In [0]:
y_valid = np.log1p(np.clip(y_valid,0,None))

In [0]:
from numba import jit
import math


In [0]:
@jit
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [0]:
def print_score(m):
    res = [smape_fast(m.predict(X_train),y_train),
           smape_fast(m.predict(X_valid),y_valid),
           m.score(X_train,y_train),
           m.score(X_valid,y_valid)]
    if hasattr(m,'oob_score_'): res.append(m.oob_score_)
    print(res)

# Random Forest

In [0]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[1.5660734106899519, 3.8484213671791614, 0.9820189407435865, 0.8915350278536259]


In [0]:
m = RandomForestRegressor(n_jobs=-1,n_estimators=50)
m.fit(X_train,y_train)
print_score(m)

[1.4554230769816594, 3.714203533285279, 0.9860304325579082, 0.898962698652027]


In [0]:
m = RandomForestRegressor(n_jobs=-1,n_estimators=50, min_samples_leaf=3)
m.fit(X_train,y_train)
print_score(m)

[2.4340094180411356, 3.575864897690154, 0.9608619098715135, 0.9065337376869694]


**Using ParameterGrid and Parfit**

In [0]:
import parfit.parfit as pf
#import numpy as np
from sklearn.model_selection import ParameterGrid
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [0]:
paramGrid = ParameterGrid({
    'min_samples_leaf': [1,3,5,10,15,25,50],
    'max_features': ['sqrt', 'log2', 0.4, 0.5, 0.6, 0.7],
    'n_estimators': [60],
    'n_jobs': [-1],
    #'random_state': [42]
})

In [0]:
best_model, best_score, all_models, all_scores = pf.bestFit(RandomForestRegressor, paramGrid, 
     X_train, y_train, X_valid, y_valid, 
     metric=roc_auc_score, greater_is_better=True, scoreLabel='AUC')
print(best_model)

-------------FITTING MODELS-------------


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.7min
exception calling callback for <Future at 0x7f93b3898da0 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 759, in dispatch_one_batch
    self._dispatch(tasks)
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 716, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/usr/local/lib/python3.6/dist-pack

TerminatedWorkerError: ignored

#Feature important


In [0]:
fi = rf_feat_importance(m,X_train); fi[:10]

,cols,imp
1,item,0.609759
0,store,0.132663
3,Month,0.089672
14,Elapsed,0.062194
6,Dayofweek,0.051603
7,Dayofyear,0.024944
5,Day,0.017855
4,Week,0.009070
2,Year,0.001153
9,Is_month_start,0.000407
